In [1]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [2]:
sleep_time = 1 # Đơn vị: giây

In [3]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

In [69]:
def get_user_playlist(user_id_file, client_id):
    
    list_user_id = np.loadtxt(user_id_file, dtype=int) # User id list
    
    # Initialize playlist dataframe
    playlist = pd.DataFrame(columns = ['id', 'title', 'genre', 'duration', 'likes_count', 'reposts_count', 'is_album', \
                                       'created_at','license', 'user_id', 'track_count', 'tracks', 'permalink_url'])
    
    # Attributes in the playlist dataframe (except tracks)
    attributes = ['id', 'title', 'genre', 'duration', 'likes_count', 'reposts_count', 'is_album', \
                                       'created_at','license', 'user_id', 'track_count', 'permalink_url']

    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[403, 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    for user_id in list_user_id:    
        # Get request with corresponding user_id
        url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}'
        print('Get:', url)
        print('Status: Doing')
        
        try:
            re = s.get(url)
        except Exception as e:
            if e != KeyboardInterrupt:
                print("Error or timeout! Status code:",re.status_code)
                print("Ignore this user")
                continue
        
        # Loop for checking if can make a request
        canGet = False
        while not canGet:
            # Sleep then make the request again
            if not re.ok:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Status: Not ok')
                continue
            
            # Check request timeout
            # If it is timeout, sleep then make the request again
            if re.status_code == 504:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Status: Timeout')
                continue
                
            if re.status_code == 429:
                print('Status: Exceeds an API rate limit')
                return playlist
                
            # Convert Json string into python dictionary
            json_pydata = json.loads(re.text)
            
            # Request is done
            canGet = True
            
        print('Status: Done')
            
        # Loop each playlist of the user
        for pl in json_pydata['collection']:
            tracks = ','.join([str(track['id']) for track in pl['tracks']]) # List of tracks in the playlist
            temp = {} # Temporary dictionary for storing the current playlist record
            
            # Get corresponding attributes into temp
            for key in attributes:
                temp[key] = pl[key]
            temp['tracks'] = tracks
            
            # Append the record into the dataframe
            playlist = playlist.append(temp, ignore_index=True)
        
    return playlist

In [74]:
playlist = get_user_playlist('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/217441590/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/324531068/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/525378972/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/9855085/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/732907996/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/311417284/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/331219188/playlists?client_id=3jXdkVwgGnCwmB

Get: https://api-v2.soundcloud.com/users/607205031/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/819670534/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/745766704/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/68715998/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/18594680/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/374261675/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/17603266/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/427999/playlists?client_id=3jXdkVwgGnCwmB9q5e

Get: https://api-v2.soundcloud.com/users/101312044/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/144263339/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/674905847/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/101768616/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/17949540/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/125999553/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/225675436/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/599232756/playlists?client_id=3jXdkVwgGnCwm

Get: https://api-v2.soundcloud.com/users/459880035/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/165028198/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/205615220/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/213880402/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/27494943/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/187357529/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/218972282/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/231255303/playlists?client_id=3jXdkVwgGnCwm

Status: Done
Get: https://api-v2.soundcloud.com/users/12218334/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/11489475/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/301904/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/332291803/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/5501103/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/386529572/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/585034569/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/179321581/playlists?client_id=3jXdkV

...

Status: Done
Get: https://api-v2.soundcloud.com/users/52019567/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/20772533/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/893883/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/389029920/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/4402819/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/752296552/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/456801519/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/39501651/playlists?client_id=3jXdkVw

Status: Done
Get: https://api-v2.soundcloud.com/users/385729871/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/78707508/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/241539717/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/956881543/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/103235250/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/336199885/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/87187738/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/37005923/playlists?client_id=3j

Status: Done
Get: https://api-v2.soundcloud.com/users/876356704/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/312392578/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/131440780/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/466290561/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/142328315/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/148737351/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/537292359/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/277214524/playlists?client_id

Status: Done
Get: https://api-v2.soundcloud.com/users/946083259/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/2492807/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/18825821/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/940944472/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/6306315/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/29989882/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/624524112/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/7533219/playlists?client_id=3jXdkVw

Status: Done
Get: https://api-v2.soundcloud.com/users/14479691/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/148501881/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/203937067/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/205015297/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/126176881/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/32503304/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/8139697/playlists?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/41691970/playlists?client_id=3jXd

In [76]:
playlist.to_csv('playlist.csv')

In [77]:
df = pd.read_csv('playlist.csv')
df.head()

,Unnamed: 0,id,title,genre,duration,likes_count,reposts_count,is_album,created_at,license,user_id,track_count,tracks,permalink_url
0,0,1215929743,N3X - D R E A M S,Pop / Tropical House,1100486,11.0,0,True,2021-02-23T13:45:56Z,all-rights-reserved,217441590,6,"991225852,991226437,991225747,991228738,991227...",https://soundcloud.com/freetousemusic/sets/n3x...
1,1,1211160604,Royalty Free Music,Royalty Free Music,9529286,6.0,0,False,2021-02-14T09:35:26Z,all-rights-reserved,217441590,52,"892455691,957116236,976211350,946099066,897495...",https://soundcloud.com/freetousemusic/sets/roy...
2,2,1211139496,Copyright Free Music,Copyright Free Music,9529286,5.0,0,False,2021-02-14T08:19:07Z,all-rights-reserved,217441590,52,"893212930,892455691,954870745,941205094,976211...",https://soundcloud.com/freetousemusic/sets/cop...
3,3,1211133652,Background Music For Videos,Background Music For Videos,9529286,6.0,1,False,2021-02-14T07:57:14Z,all-rights-reserved,217441590,52,"941194555,933862858,933856318,967357153,892455...",https://soundcloud.com/freetousemusic/sets/bac...
4,4,1211125735,No Copyright Music,No Copyright Music,9529286,6.0,0,False,2021-02-14T07:29:15Z,all-rights-reserved,217441590,52,"976211350,970438177,967357153,957116236,967224...",https://soundcloud.com/freetousemusic/sets/no-...
